Eval scripts adapted from https://github.com/SpaceNetChallenge/utilities/tree/master/python

In [ ]:
import numpy as np
import geopandas as gpd
import rtree

from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

from tqdm import tqdm

In [ ]:
def create_rtree_from_poly(poly_list):
    # create index
    index = rtree.index.Index(interleaved=False)
    for idx, building in enumerate(poly_list):
        minx, miny, maxx, maxy = building.bounds
        envelope = (minx, maxx, miny, maxy)
        index.insert(idx, envelope)

    return index

def search_rtree(test_building, index):
    # input test poly ogr.Geometry  and rtree index
    if test_building.type == 'Polygon' or \
                    test_building.type == 'MultiPolygon':
        minx, miny, maxx, maxy = test_building.bounds
        envelope = (minx, maxx, miny, maxy)    
        fidlist = index.intersection(envelope)
    else:
        fidlist = []

    return fidlist


In [ ]:
def iou(test_poly, truth_polys, truth_index=[]):
    fidlistArray = []
    iou_list = []
    
    if truth_index:
        fidlist = search_rtree(test_poly, truth_index)

        for fid in fidlist:
            if not test_poly.is_valid:
                test_poly = test_poly.buffer(0.0)

            intersection_result = test_poly.intersection(truth_polys[fid].buffer(0.0))
            fidlistArray.append(fid)

            if intersection_result.type == 'Polygon' or \
                            intersection_result.type == 'MultiPolygon':
                intersection_area = intersection_result.area
                union_area = test_poly.union(truth_polys[fid].buffer(0.0)).area
                iou_list.append(intersection_area / union_area)

            else:
                iou_list.append(0)

    else:
        for idx, truth_poly in enumerate(truth_polys):
            if not test_poly.is_valid or not truth_poly.is_valid:
                test_poly = test_poly.buffer(0.0)
                truth_poly = truth_poly.buffer(0.0)
#                 print(f'fixed geom error at {idx}')

            intersection_result = test_poly.intersection(truth_poly)
            #print(idx, intersection_result.type)

            if intersection_result.type == 'Polygon' or \
                            intersection_result.type == 'MultiPolygon':
                intersection_area = intersection_result.area
                union_area = test_poly.union(truth_poly).area
                iou_list.append(intersection_area / union_area)
    #             print(f'found intersect at test_poly {i} with truth poly {idx}')
    #             print(intersection_area/union_area)
            else:    
                iou_list.append(0)
                
    return iou_list, fidlistArray

In [ ]:
def score(test_polys, truth_polys, threshold=0.5, truth_index=[],
          resultGeoJsonName = [],
          imageId = []):

    # Define internal functions

    # Find detections using threshold/argmax/IoU for test polygons
    true_pos_count = 0
    false_pos_count = 0
    truth_poly_count = len(truth_polys)
    
    true_ids = []
    false_ids = []

    for idx, test_poly in tqdm(enumerate(test_polys)):
        if truth_polys:
            iou_list, fidlist = iou(test_poly, truth_polys, truth_index)
            if not iou_list:
                maxiou = 0
            else:
                maxiou = np.max(iou_list)

#             print(maxiou, iou_list, fidlist)
            if maxiou >= threshold:
                true_pos_count += 1
                true_ids.append(idx)
                minx, miny, maxx, maxy = truth_polys[fidlist[np.argmax(iou_list)]].bounds
                envelope = (minx, maxx, miny, maxy)   
                truth_index.delete(fidlist[np.argmax(iou_list)], envelope)
                #del truth_polys[fidlist[np.argmax(iou_list)]]
            else:
                false_pos_count += 1
                false_ids.append(idx)
        else:
            false_pos_count += 1
            false_ids.append(idx)

    false_neg_count = truth_poly_count - true_pos_count

    return true_pos_count, false_pos_count, false_neg_count, true_ids, false_ids

In [ ]:
def evalfunction(image_id, test_polys, truth_polys, truth_index=[], resultGeoJsonName=[], threshold = 0.5):

    if len(truth_polys)==0:
        true_pos_count = 0
        false_pos_count = len(test_polys)
        false_neg_count = 0
    else:
        true_pos_count, false_pos_count, false_neg_count, true_ids, false_ids = score(test_polys, truth_polys,
                                                                 truth_index=truth_index,
                                                                 resultGeoJsonName=resultGeoJsonName,
                                                                 imageId=image_id,
                                                                 threshold=threshold
                                                                 )


    if (true_pos_count > 0):

        precision = float(true_pos_count) / (float(true_pos_count) + float(false_pos_count))
        recall = float(true_pos_count) / (float(true_pos_count) + float(false_neg_count))
        F1score = 2.0 * precision * recall / (precision + recall)
    else:
        F1score = 0
    return ((F1score, true_pos_count, false_pos_count, false_neg_count), true_ids, false_ids, image_id)

In [ ]:
def precision_recall(true_pos_count, false_pos_count, false_neg_count):
    precision = float(true_pos_count) / (float(true_pos_count) + float(false_pos_count))
    recall = float(true_pos_count) / (float(true_pos_count) + float(false_neg_count))
    return (precision, recall)

In [ ]:
TRUTH = Path('znz-input')
TEST = Path('znz-20190118')

In [ ]:
df_truth = gpd.read_file(f'{str(TRUTH)}/grid_042.geojson')
df_test = gpd.read_file(f'{str(TEST)}/grid_042_20190118_07_classes.geojson')

In [ ]:
df_truth.head()

In [ ]:
df_test.head()

In [ ]:
df_truth.geometry.plot(figsize=(10,10))

In [ ]:
df_test.geometry.plot(figsize=(10,10))

In [ ]:
df_test['cat'].value_counts()

In [ ]:
df_truth['condition'].value_counts()

In [ ]:
cats = [('conf_foundation','Foundation'),('conf_unfinished','Incomplete'),('conf_completed','Complete')]

In [ ]:
for (test_cat, truth_cat) in cats:
    test_polys = [geom for geom in df_test[df_test['cat'] == test_cat].geometry]
    truth_polys = [geom for geom in df_truth[df_truth['condition'] == truth_cat].geometry]
    truth_index = create_rtree_from_poly(truth_polys)
    scores = evalfunction(grid_num,test_polys, truth_polys, truth_index=truth_index)
    print(truth_cat)
    print(scores[0],precision_recall(*scores[0][1:]))

In [ ]:
test_polys = [geom for geom in df_test.geometry]
truth_polys = [geom for geom in df_truth.geometry]
truth_index = create_rtree_from_poly(truth_polys)

In [ ]:
scores = evalfunction(grid_num,test_polys, truth_polys, truth_index=truth_index)
scores[0],precision_recall(*scores[0][1:])